# Lemmatizing working abstracts

The text has already been cleaned.  This script will lemmatize those new working abstracts from the new cleaned df that were not in the old cleaned df.

**Note**: There is an important change in order from past runs of the code here. Now, lemmatization and tokenization occurs within the same step.  Some clean up including stop word removal and the creation of n-grams occurs AFTER lemmatization.

In [1]:
import pandas as pd
import pickle
import numpy as np
import time
import matplotlib.pyplot as plt

import stanza
import spacy
from spacy.lang.en.stop_words import STOP_WORDS
#import gensim
from gensim.models.phrases import Phrases, Phraser
import re

pd.set_option('display.max_columns', 50)

/home/kjl5t/.local/lib/python3.8/site-packages/gensim/similarities/__init__.py:15: UserWarning: The gensim.similarities.levenshtein submodule is disabled, because the optional Levenshtein package <https://pypi.org/project/python-Levenshtein/> is unavailable. Install Levenhstein (e.g. `pip install python-Levenshtein`) to suppress this warning.
  warnings.warn(msg)


### Read in Clean Datasets - old: df, new: df_new

In [2]:
# load saved df.  df['working_abstract'] contains cleaned text.

df = pd.read_pickle("/home/kjl5t/dspg20RnD/data/working/clean_data_7-20.pkl")
df.reset_index(inplace = True)
df.rename(columns={'index':'original index'}, inplace=True)

wa='working_abstract'

In [3]:
df.head()

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR
0,17608,152242,The multiprotein complex y-secretase proteolyt...,2008,Active Sites; Affect; Alzheimer's Disease; Amy...,STRUCTURE OF SIGNAL PEPTIDE PEPTIDASE,HHS,NIH,NaN,5F32AG027647-03,12/1/2005,1/1/2008,"LIEBERMAN, RAQUEL L",NaN,07,030811269,BRIGHAM AND WOMEN'S HOSPITAL,BOSTON,MA,021156110,UNITED STATES,NaN,NaN,93.866,2008,3483.0,NaN,1,1,The multiprotein complex y-secretase proteolyt...,T,1402,g
1,111864,190316,DESCRIPTION (provided by applicant): The Kis...,2008,Affect; Animal Model; Axon; Behavior; Behavior...,ROLE OF KISS1 NEURONS IN THE SEASONAL AND CIRC...,HHS,NIH,NaN,5K99HD056157-02,9/1/2007,1/1/2009,"KAUFFMAN, ALEXANDER S",NaN,07,605799469,UNIVERSITY OF WASHINGTON,SEATTLE,WA,981959472,UNITED STATES,NaN,NaN,93.865,2008,39175.0,NaN,1,1,The Kissl gene encodes peptides called kisspep...,T,2553,y
2,22052,154213,DESCRIPTION (provided by applicant): The objec...,2008,Agreement; Antibodies; base; Binding; Biochemi...,CARBONIC ANHYDRASE AS A MODEL TO UNDERSTAND DI...,HHS,NIH,NaN,5F32GM076971-02,1/2/2007,1/1/2009,"MACK, ERIC T",NaN,05,082359691,HARVARD UNIVERSITY,CAMBRIDGE,MA,021385319,UNITED STATES,NaN,NaN,93.859,2008,49646.0,NaN,1,1,The objective of this research is to understan...,T,1414,e
3,35004,159362,Obesity is the cause of many adverse pregnancy...,2008,African; Analysis of Variance; Asians; Birth; ...,OBESITY ON VAGAL TONE AND HBA1C DURING PREGNANCY,HHS,NIH,NaN,5F31NR009611-03,4/1/2006,1/1/2009,"HELMREICH, REBECCA J",NaN,09,800771594,UNIVERSITY OF TEXAS HLTH SCI CTR HOUSTON,HOUSTON,TX,770305400,UNITED STATES,NaN,NaN,93.361,2008,20406.0,NaN,1,1,Obesity is the cause of many adverse pregnancy...,O,1545,d
4,371628,594482,Local potato advisory groups have expressed in...,2010,cost; Health; interest; Manure; Parasitic nema...,PLANT-PARASITIC NEMATODE MANAGEMENT AS A COMPO...,USDA,NIFA,NaN,0219605,10/1/2009,1/1/2010,"CHARLTON, B.",NaN,NaN,053599908,OREGON STATE UNIVERSITY,CORVALLIS,OR,97331,UNITED STATES,NaN,NaN,10.203,2010,NaN,NaN,1,1,Local potato advisory groups have expressed in...,L,271,s


In [4]:
# load saved df.  df['working_abstract'] contains cleaned text.

df_new = pd.read_pickle("../../../data/prd/Tech-Report/FR_clean_21SEPT07.pkl")
df_new.reset_index(inplace = True)
df_new.rename(columns={'index':'original index'}, inplace=True)

wa='working_abstract'

In [5]:
df_new.head()

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR
0,0,89996,"This is a project to explore Game-based, Metap...",2008,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,"This is a project to explore Game-based, Metap...",T,2057,T
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,Institution: Franklin Institute Science Museum...,I,2053,I
2,2,89998,Through programs (including small group conver...,2008,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,Through programs (including small group conver...,T,1154,T
3,3,89999,In partnership with the American Chemical Soci...,2008,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,In partnership with the American Chemical Soci...,I,875,I
4,5,90001,The Center for Molecular Interfacing (CMI) wil...,2008,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,The Center for Molecular Interfacing (CMI) wil...,T,2309,T


In [6]:
print(len(df))
print(len(df_new))

690855
696128


## Find out which working abstracts in df_new need to be lemmatized. Exploration.

** can be skipped in later code runs **

- Projects not already in df
- Projects in df with a different working abstract.

In [8]:
print(len(pd.unique(df['working_abstract'])))
print(len(pd.unique(df_new['working_abstract'])))

565732
569053


In [9]:
# compare project IDs

A = set(df["PROJECT_ID"])
B = set(df_new["PROJECT_ID"])

In [10]:
print(len(A.difference(B))) # don't need these
print(len(B.difference(A)))  # need to lemmatize these -- only unique cases

83475
88748


In [11]:
ids_in_both = list(A.intersection(B))
len(ids_in_both)

607380

In [12]:
ids_in_both[1:10]

['415635',
 '74447',
 '1117035',
 '112837',
 '1173676',
 '1019458',
 '835736',
 '361208',
 '132518']

In [13]:
df[df['PROJECT_ID'] == '158670']

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR
90311,33591,158670,DESCRIPTION (provided by applicant): The aggr...,2008,alpha synuclein; analog; Animal Model; base; C...,CATECHOL-INDUCED INHIBITION OF ALPHA-SYNUCLEIN...,HHS,NIH,NaN,5R01NS048584-04,6/1/2005,2/28/2010,"MILLHAUSER, GLENN L",NaN,20,...,CA,950641077,UNITED STATES,NaN,NaN,93.853,2008,282793.0,NaN,1,1,The aggregation/fibrillation of alpha-synuclei...,T,1903,e


In [14]:
df[wa][90311]

"The aggregation/fibrillation of alpha-synuclein is a critical factor in Parkinson's disease (PD). Alpha-Synuclein is a significant component of intracellular inclusions known as Lewy bodies that are the pathological hallmark of PD and mutations and gene triplication of alpha-synuclein have been associated with rare cases of familial PD. Our overall goals are to design molecules that will inhibit alpha-synuclein aggregation, and disaggregate existing alpha-synuclein fibrils. There is a critical need for an effective treatment of Parkinson's disease, since current therapies are only partially effective in treating the symptoms. Our preliminary data indicate that some catechols, flavonoids and related compounds have the potential to inhibit fibrillation of alpha-synuclein and disaggregate existing fibrils.  The aims of this proposal are:  1) To determine the underlying molecular mechanisms of the inhibition of alpha-synuclein fibrillation by catechols, flavonoids and related compounds. O

In [15]:
df_new[df_new['PROJECT_ID'] == '158670']

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,...,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR
16038,22255,158670,DESCRIPTION (provided by applicant): The aggr...,2008,alpha synuclein; analog; Animal Model; base; C...,CATECHOL-INDUCED INHIBITION OF ALPHA-SYNUCLEIN...,HHS,NIH,NINDS,5R01NS048584-04,6/1/2005,2/28/2010,"MILLHAUSER, GLENN L",NaN,20,...,CA,950641077,UNITED STATES,3/1/2008,2/28/2010,93.853,2008,282793.0,NaN,1,1,The aggregation/fibrillation of alpha-synuclei...,T,1942,T


In [39]:
# merge in df[wa] column with df_new to compare which working abstracts are different

df_both_wa = df_new[df_new["PROJECT_ID"].isin(ids_in_both)]

In [44]:
df = df.rename(columns={'working_abstract': 'old_working_abstract'})
df_both_wa = df_both_wa.merge(df[['PROJECT_ID', 'old_working_abstract']], how='left', on='PROJECT_ID')

In [48]:
df_both_wa.head()

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR,old_working_abstract
0,0,89996,"This is a project to explore Game-based, Metap...",2008,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,"This is a project to explore Game-based, Metap...",T,2057,T,"This is a project to explore Game-based, Metap..."
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,Institution: Franklin Institute Science Museum...,I,2053,I,"Institution: Science Museum PI: Snyder, Steve..."
2,2,89998,Through programs (including small group conver...,2008,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,Through programs (including small group conver...,T,1154,T,Through programs (including small group conver...
3,3,89999,In partnership with the American Chemical Soci...,2008,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,In partnership with the American Chemical Soci...,I,875,I,In partnership with the American Chemical Soci...
4,5,90001,The Center for Molecular Interfacing (CMI) wil...,2008,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,The Center for Molecular Interfacing (CMI) wil...,T,2309,T,The Center for Molecular Interfacing (CMI) wil...


In [50]:
s = sum(df_both_wa[wa] == df_both_wa["old_working_abstract"])
print(len(df_both_wa) - s)

101975


In [53]:
df_need_lemma = df_both_wa[df_both_wa[wa] != df_both_wa["old_working_abstract"]]

In [54]:
len(df_need_lemma)

101975

In [64]:
print(df_need_lemma[wa].iloc[1500])
print('\n')
print(df_need_lemma["old_working_abstract"].iloc[1500])

This award, co-funded by the Division of Materials Research and the Office of International Science and Engineering, aims at the synthesis and characterization of novel self-assembling nanostructured thermoplastic elastomeric (TPE) biomaterials based on polyisobutylene (PIB).  Specifically, efforts to synthesize new  Entropy-driven  TPEs (ENTPE) will continue. Under the current NSF support it was discovered that the dendritic (arborescent or tree-like) arbPIB midblock of ENTPEs facilitates phase separation and TPE behavior even with very short plastic and elastomeric end blocks.  These novel ENTPE materials were further reinforced with nano-size fillers (carbon and silica) to yield strong rubbery nanocomposites (ENTPEC), and their biocompatibility was demonstrated in vitro and in vivo in rabbits. Based on this discovery, new ENTPEs will be synthesized with short end blocks, capable of hydrogen bonding (e.g., polypeptides). Compounding these materials with nanofillers will yield new ENT

In [65]:
len(pd.unique(df_need_lemma[wa]))

70663

In [67]:
ids_new = list(B.difference(A))

In [69]:
df_ids_new = df_new[df_new["PROJECT_ID"].isin(ids_new)]

In [72]:
len(pd.unique(df_ids_new[wa]))

87358

In [ ]:
df[wa]

In [76]:
# compare project wa's

A = set(df[wa])
B = set(df_new[wa])

In [77]:
len(B.difference(A))

82990

#### Plan: 
- lemmatize those working abstracts that are not in the old cleaned dataset.      
- Then merge these in with results already completed on the projects that are in both the new and old dataset.

In [7]:
# find project id's for those with a working abstract that is not in the old working abstract set

old_wa = set(df[wa])

In [8]:
idx = []

for i in range(len(df_new)):
    if df_new[wa].iloc[i] not in old_wa:
        idx.append(i)

In [9]:
len(idx)

115961

In [10]:
idx[0:10]

[1, 3, 5, 6, 14, 23, 26, 35, 41, 53]

In [11]:
df_need_lemmas = df_new.iloc[idx]

In [12]:
len(df_need_lemmas)

115961

In [13]:
len(pd.unique(df_need_lemmas[wa]))

82990

In [14]:
df_to_lemmatizer = pd.DataFrame(pd.unique(df_need_lemmas[wa]), columns = ['working_abstract'])

In [15]:
df_to_lemmatizer.to_pickle('abs_to_lemmatize.pkl')

In [16]:
df_to_lemmatizer.head()

,working_abstract
0,Institution: Franklin Institute Science Museum...
1,In partnership with the American Chemical Soci...
2,DRU: Integrated optimization of evacuation and...
3,The Flora of China (FOC) is an international c...
4,This multidisciplinary project draws upon soci...


### Tokenize, find POS, and Lemmatize -   

**done in 02-lemmatize.py so it could be ported as a SLURM script and use GPU.**

In [7]:
# read in data from 02-lemmatize.py

df_new_lemmas = pd.read_pickle("lemmatized_abs.pkl")

In [8]:
df_new_lemmas.head()

,working_abstract,lemma_abstract
0,Institution: Franklin Institute Science Museum...,"[institution, Franklin, Institute, science, Mu..."
1,In partnership with the American Chemical Soci...,"[partnership, American, Chemical, Society, ACS..."
2,DRU: Integrated optimization of evacuation and...,"[DRU, integrate, optimization, evacuation, she..."
3,The Flora of China (FOC) is an international c...,"[Flora, China, FOC, international, collaborati..."
4,This multidisciplinary project draws upon soci...,"[multidisciplinary, project, draw, sociology, ..."


In [9]:
len(df_new_lemmas)

82990

### Merge in new lemmas with existing data

In [10]:
# merge new lemmas into df_new

df_new = df_new.merge(df_new_lemmas, how='left', on='working_abstract')

In [11]:
len(df_new)

696128

In [12]:
sum(~df_new['lemma_abstract'].isna())

115961

In [13]:
# read in lemma data from old cleaned data

df_old_lemmas = pd.read_pickle("/home/kjl5t/dspg20RnD/data/working/clean_data_with_lemmas_7-20.pkl")

In [14]:
df_old_lemmas_unique = df_old_lemmas[['working_abstract', 'lemma_abstract']]

In [15]:
df_old_lemmas_unique = df_old_lemmas_unique.drop_duplicates(subset=['working_abstract'])

In [16]:
len(df_old_lemmas_unique)

565732

In [17]:
# get lemmas from old dataset for those working abstracts that were in the old dataset

df_new = df_new.merge(df_old_lemmas_unique, how='left', on='working_abstract')

In [18]:
len(df_new)

696128

In [19]:
sum(df_new['lemma_abstract_y'].isna())

115961

In [20]:
df_new['lemma_abstract'] = df_new["lemma_abstract_x"].fillna(df_new['lemma_abstract_y'])

In [21]:
df_new = df_new.drop(columns = ['lemma_abstract_x', 'lemma_abstract_y'])

In [22]:
df_new.head()

,original index,PROJECT_ID,ABSTRACT,FY,PROJECT_TERMS,PROJECT_TITLE,DEPARTMENT,AGENCY,IC_CENTER,PROJECT_NUMBER,PROJECT_START_DATE,PROJECT_END_DATE,CONTACT_PI_PROJECT_LEADER,OTHER_PIS,CONGRESSIONAL_DISTRICT,DUNS_NUMBER,ORGANIZATION_NAME,ORGANIZATION_CITY,ORGANIZATION_STATE,ORGANIZATION_ZIP,ORGANIZATION_COUNTRY,BUDGET_START_DATE,BUDGET_END_DATE,CFDA_CODE,FY.y,FY_TOTAL_COST,FY_TOTAL_COST_SUB_PROJECTS,ORG_COUNT,PI_COUNT,working_abstract,Start_Char,nchar,LAST_CHAR,lemma_abstract
0,0,89996,"This is a project to explore Game-based, Metap...",2008,Achievement; analog; base; Cognitive Science; ...,RUI: CYGAMES: CYBER-ENABLED TEACHING AND LEARN...,NSF,NSF,NaN,0814512,9/15/2008,8/31/2012,"REESE, DEBBIE D","CARTER, BEVERLY; WOOD, CHARLES; HITT, BEN",01,068719400,WHEELING JESUIT UNIVERSITY,WHEELING,WV,26003-6243,UNITED STATES,NaN,NaN,47.076,2008,1999467.0,NaN,1,1,"This is a project to explore Game-based, Metap...",T,2057,T,"[project, explore, game, base, metaphor, enhan..."
1,1,89997,Institution: Franklin Institute Science Museum...,2008,Active Learning; Child; Computer software; des...,ARIEL - AUGMENTED REALITY FOR INTERPRETIVE AND...,NSF,NSF,NaN,0741659,9/15/2008,8/31/2012,"SNYDER, STEVEN","ELINICH, KAREN; YOON, SUSAN",02,001741859,FRANKLIN INSTITUTE,PHILADELPHIA,PA,19103-1115,UNITED STATES,NaN,NaN,47.076,2008,1799699.0,NaN,1,1,Institution: Franklin Institute Science Museum...,I,2053,I,"[institution, Franklin, Institute, science, Mu..."
2,2,89998,Through programs (including small group conver...,2008,Address; Age; Birth; Brain; Caregivers; Child;...,BRIGHTER FUTURES: PUBLIC DELIBERATION ABOUT TH...,NSF,NSF,NaN,0813522,9/15/2008,8/31/2011,"FINK, LAURIE KLEINBAUM","CADIGAN, KAREN; ELLENBOGEN, KIRSTEN",04,061451670,SCIENCE MUSEUM OF MINNESOTA,SAINT PAUL,MN,55102-1202,UNITED STATES,NaN,NaN,47.076,2008,1505858.0,NaN,1,1,Through programs (including small group conver...,T,1154,T,"[program, include, small, group, conversation,..."
3,3,89999,In partnership with the American Chemical Soci...,2008,Advanced Development; American; Chemicals; Che...,FOSTERING US-INTERNATIONAL COLLABORATIVE PARTN...,NSF,NSF,NaN,0838627,8/1/2008,12/31/2010,"JOST, JOHN W","MILLER, BRADLEY; BOWMAN, KATHERINE",04,009059242,INTERNATIONAL UNION OF PURE AND APPLIED CHEMISTRY,DURHAM,NC,27709-3757,UNITED STATES,NaN,NaN,47.049,2008,51000.0,NaN,1,1,In partnership with the American Chemical Soci...,I,875,I,"[partnership, American, Chemical, Society, ACS..."
4,5,90001,The Center for Molecular Interfacing (CMI) wil...,2008,Address; Architecture; Carbon Nanotubes; Catal...,CCI PHASE I: CENTER FOR MOLECULAR INTERFACING,NSF,NSF,NaN,0847926,10/1/2008,9/30/2011,"ABRUNA, HECTOR D",NaN,22,872612445,CORNELL UNIVERSITY ITHACA,ITHACA,NY,14850-2820,UNITED STATES,NaN,NaN,47.049,2008,1519821.0,NaN,1,1,The Center for Molecular Interfacing (CMI) wil...,T,2309,T,"[Center, Molecular, Interfacing, CMI, enable, ..."


In [23]:
sum(df_new['lemma_abstract'].isna())

0

In [24]:
len(df_new)

696128

In [25]:
df_new.to_pickle("../../../data/prd/Tech-Report/FR_lemmatized_21SEPT14.pkl")

### Scratch work

Stanza docs say it is more efficient to put all documents into one file and then lemmatize.  Tried this.  It offered a very minor speed up on the cpu and gpu, but not enough to be worth the processing.

In [6]:
doc_all_list = list(df_new[wa][1:500])
doc_all_list

["Institution: Franklin Institute Science Museum PI: Snyder, Steve Project: ARIEL - Augmented Reality for Interpretive and Experiential Learning Proposal No.: DRL-0741659 PROJECT SUMMARY In partnership with the University of Pennsylvania's Graduate School of Education and Carnegie Mellon University's Entertainment Technology Center, the Franklin Institute Science Museum will develop, test, and pilot an exportable and replicable cyberlearning exhibit using two cutting edge technologies: Augmented Reality (AR) and Virtual Reality (VR). The exhibit's conceptualization is anchored in the learning research vision of the NSF-funded workshop Cyberinfrastructure for Education and Learning for the Future (Computing Research Association, 2005).  The incorporation of VR and AR technologies into the Franklin Institute's electricity and Earth science exhibits is an innovation of traditional approaches to hands-on learning and will improve the quality of the learning experience for the primary audie

In [7]:
doc_all = ' NEW_ABSTRACT_MARKER '.join(doc_all_list)

In [8]:
doc_all

"Institution: Franklin Institute Science Museum PI: Snyder, Steve Project: ARIEL - Augmented Reality for Interpretive and Experiential Learning Proposal No.: DRL-0741659 PROJECT SUMMARY In partnership with the University of Pennsylvania's Graduate School of Education and Carnegie Mellon University's Entertainment Technology Center, the Franklin Institute Science Museum will develop, test, and pilot an exportable and replicable cyberlearning exhibit using two cutting edge technologies: Augmented Reality (AR) and Virtual Reality (VR). The exhibit's conceptualization is anchored in the learning research vision of the NSF-funded workshop Cyberinfrastructure for Education and Learning for the Future (Computing Research Association, 2005).  The incorporation of VR and AR technologies into the Franklin Institute's electricity and Earth science exhibits is an innovation of traditional approaches to hands-on learning and will improve the quality of the learning experience for the primary audien

In [ ]:
t1 = time.time()
lems = token_pos_lemma(doc_all)
t2 = time.time()
print(t2-t1)